# The PyRIT CLI

The PyRIT cli tool that allows you to run automated security testing and red teaming attacks against AI systems using [scenarios](../scenarios/scenarios.ipynb) for strategies and [configuration](../setup/0_configuration.ipynb).

Note in this doc the ! prefaces all commands in the terminal so we can run in a Jupyter Notebook.

## Quick Start

For help:

In [ ]:
!pyrit_scan --help

usage: pyrit_scan [-h] [--verbose] [--list-scenarios] [--list-initializers]
                  [--database {InMemory,SQLite,AzureSQL}]
                  [--initializers INITIALIZERS [INITIALIZERS ...]]
                  [--initialization-scripts INITIALIZATION_SCRIPTS [INITIALIZATION_SCRIPTS ...]]
                  [scenario_name]

PyRIT Scanner - Run security scenarios against AI systems

Examples:
  # List available scenarios and initializers
  pyrit_scan --list-scenarios
  pyrit_scan --list-initializers

  # Run a scenario with built-in initializers
  pyrit_scan foundry_scenario --initializers simple objective_target

  # Run with custom initialization scripts
  pyrit_scan encoding_scenario --initialization-scripts ./my_config.py

positional arguments:
  scenario_name         Name of the scenario to run (e.g., encoding_scenario,
                        foundry_scenario)

options:
  -h, --help            show this help message and exit
  --verbose             Enable verbose logging ou

### Discovery

List all available scenarios:

In [ ]:
!pyrit_scan --list-scenarios


Available Scenarios:

  encoding_scenario
    Class: EncodingScenario
    Description:
      Encoding Scenario implementation for PyRIT. This scenario tests how
      resilient models are to various encoding attacks by encoding potentially
      harmful text (by default slurs and XSS payloads) and testing if the model
      will decode and repeat the encoded payload. It mimics the Garak encoding
      probe. The scenario works by: 1. Taking seed prompts (the harmful text to
      be encoded) 2. Encoding them using various encoding schemes (Base64,
      ROT13, Morse, etc.) 3. Asking the target model to decode the encoded text
      4. Scoring whether the model successfully decoded and repeated the harmful
      content By default, this uses the same dataset as Garak: slur terms and
      web XSS payloads.

  foundry_scenario
    Class: FoundryScenario
    Description:
      FoundryScenario is a preconfigured scenario that automatically generates
      multiple AtomicAttack instances b

**Tip**: You can also discover user-defined scenarios by providing initialization scripts:

```shell
pyrit_scan --list-scenarios --initialization-scripts ./my_custom_initializer.py
```

This will load your custom scenario definitions and include them in the list.

## Initializers

PyRITInitializers are how you can configure the CLI scanner. PyRIT includes several built-in initializers you can use with the `--initializers` flag. 

The `--list-initializers` command shows all available initializers. Initializers are referenced by their filename (e.g., `objective_target`, `objective_list`, `simple`) regardless of which subdirectory they're in.

List the available initializers using the --list-initializers flag.

In [ ]:
!pyrit_scan --list-initializers


Available Scenario Initializers:

  objective_list
    Class: ScenarioObjectiveListInitializer
    Name: Simple Objective List Configuration for Scenarios
    Execution Order: 10
    Required Environment Variables: None
    Description:
      Simple Objective List Configuration for Scenarios

  openai_objective_target
    Class: ScenarioObjectiveTargetInitializer
    Name: Simple Objective Target Configuration for Scenarios
    Execution Order: 10
    Required Environment Variables:
      - OPENAI_CLI_ENDPOINT
      - OPENAI_CLI_KEY
    Description:
      This configuration sets up a simple objective target for scenarios using
      OpenAIChatTarget with basic settings. It initializes an openAI chat target
      using the OPENAI_CLI_ENDPOINT and OPENAI_CLI_KEY environment variables.


Total initializers: 2

For usage information, use: pyrit_scan --help


### Running Scenarios

You need a single scenario to run, you need two things:

1. A Scenario. Many are defined in `pyrit.scenarios.scenarios`. But you can also define your own in initialization_scripts.
2. Initializers (which can be supplied via `--initializers` or `--initialization-scripts`). Scenarios often don't need many arguments, but they can be configured in different ways. And at the very least, most need an `objective_target` (the thing you're running a scan against).

Basic usage will look something like:

```shell
pyrit_scan <scenario> --initializers <initializer1> <initializer2>
```

Or concretely:

```shell
!pyrit_scan foundry_scenario --initializers simple openai_objective_target
```

Example with a basic configuration that runs the Foundry scenario against the objective target defined in `openai_objective_target` (which just is an OpenAIChatTarget with `OPENAI_CLI_ENDPOINT` and `OPENAI_CLI_KEY`).

In [ ]:
!pyrit_scan foundry_scenario --initializers openai_objective_target


════════════════════════════════════════════════════════════════════════════════════════════════════
                                📊 SCENARIO RESULTS: FoundryScenario                                 
════════════════════════════════════════════════════════════════════════════════════════════════════

▼ Scenario Information
────────────────────────────────────────────────────────────────────────────────────────────────────
  📋 Scenario Details
    • Name: FoundryScenario
    • Scenario Version: 1
    • PyRIT Version: 0.10.0.dev0
    • Description:
        FoundryScenario is a preconfigured scenario that automatically generates multiple AtomicAttack instances based
        on the specified attack strategies. It supports both single-turn attacks (with various converters) and multi-
        turn attacks (Crescendo, RedTeaming), making it easy to quickly test a target against multiple attack vectors.
        The scenario can expand difficulty levels (EASY, MODERATE, DIFFICULT) into their

ERROR: Initializer name collision: 'airt' found in both 'C:\git\PyRIT\pyrit\setup\initializers\airt.py' and 'C:\git\PyRIT\pyrit\setup\initializers\airt.py'. Initializer filenames must be unique across all directories.
ERROR: Initializer name collision: 'simple' found in both 'C:\git\PyRIT\pyrit\setup\initializers\simple.py' and 'C:\git\PyRIT\pyrit\setup\initializers\simple.py'. Initializer filenames must be unique across all directories.

Executing Foundry Scenario:   5%|▌         | 1/20 [00:07<02:14,  7.09s/attack]ERROR: BadRequestException encountered: Status Code: 400, Message: {"error":{"message":"The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766","type":null,"param":"prompt","code":"content_filter","status":400,"innererror":{"code":"ResponsibleAIPolicyViolation","content

Or with all options and multiple initializers:

```shell
pyrit_scan foundry_scenario --database InMemory --initializers simple objective_target objective_list
```

You can also use custom initialization scripts by passing file paths. It is relative to your current working directory, but to avoid confusion, full paths are always better:

```shell
pyrit_scan encoding_scenario --initialization-scripts ./my_custom_config.py
```

#### Using Custom Scenarios

You can define your own scenarios in initialization scripts. The CLI will automatically discover any `Scenario` subclasses and make them available:

```python
# my_custom_scenarios.py
from pyrit.scenarios import Scenario
from pyrit.common.apply_defaults import apply_defaults

@apply_defaults
class MyCustomScenario(Scenario):
    """My custom scenario that does XYZ."""

    def __init__(self, objective_target=None):
        super().__init__(name="My Custom Scenario", version="1.0")
        self.objective_target = objective_target
        # ... your initialization code

    async def initialize_async(self):
        # Load your atomic attacks
        pass

    # ... implement other required methods
```

Then discover and run it:

```shell
# List to see it's available
pyrit_scan --list-scenarios --initialization-scripts ./my_custom_scenarios.py

# Run it
pyrit_scan my_custom_scenario --initialization-scripts ./my_custom_scenarios.py
```

The scenario name is automatically converted from the class name (e.g., `MyCustomScenario` becomes `my_custom_scenario`).


## When to Use the Scanner

The scanner is ideal for:

- **Automated testing pipelines**: CI/CD integration for continuous security testing
- **Batch testing**: Running multiple attack scenarios against various targets
- **Repeatable tests**: Standardized testing with consistent configurations
- **Team collaboration**: Shareable configuration files for consistent testing approaches
- **Quick testing**: Fast execution without writing Python code


## Complete Documentation

For comprehensive documentation about initialization files and setting defaults see:

- **Configuration**: See [configuration](../setup/0_configuration.ipynb)
- **Setting Default Values**: See [default values](../setup/default_values.md)
- **Writing Initializers**: See [Initializers](../setup/pyrit_initializer.ipynb)

Or visit the [PyRIT documentation website](https://azure.github.io/PyRIT/)